# Chemical Reaction Network Reduction

## Set Up

In [1]:
#import necessary packages
import numpy as np
import sys
import math
import re
import isoprene_rates as rate
from math import exp as EXP
from copy import deepcopy
import sympy as sym
import networkx as nx
import matplotlib.pyplot as plt
import graphviz 
#import pygraphviz as pgv
#import to_precision
import time
import csv
import pandas as pd
import random
from sklearn.preprocessing import normalize


In [2]:
#KPP eqns to python (rxn,rate) format
#as an input and outputs the reactions as a list in the format of [[reaction, rate],[r2],[r3]...]
def read_eqns(eqn_file):
    '''Read .eqn files
    Parameters
    ----------
    eqn_file: .eqn file
      The .eqn file to read

    Returns
    ----------
    species: list
      A list of tuples. The first element in the tuple is an equation.
      The second element in the tuple is reaction rate. 
    '''
  
    equations = None
    with open(eqn_file,'r') as f:
        lines = f.readlines()
        equantions = lines[:]
    equations = [i.strip() for i in equantions[1:]]
    equations = [tuple(i.split(':')) for i in equations if len(i)>0]
    equations = [(i[0].strip(),i[1].strip().strip(';')) for i in equations]
    return(equations)

In [3]:
#KPP spc to python 
#as an input and outputs the reactions as a list in the format of [species1,species2,...]
def read_spc(spc_file):
    '''Read .spc files and process the raw input into species
    Parameters
    ----------
    spc_file: .eqn file
        The .spc file to read
    
    Returns
    ----------
    species: list
        List of species.
    '''
    species = None
    with open(spc_file,'r') as f:
        lines = f.readlines()
        species = lines[:]
    species = [s.split('=')[0].strip() for s in species]
    species = [s for s in species if s and s[0]!='#']
    return(species)

In [4]:
#python functions for all of the rate constant functions given in the mechanism file. 
#These were copied from isoprene_rates.py, made by the DSI team
def ISO1(A0, B0, C0, D0, E0, F0, G0):
    K0 = D0 * EXP(E0/TEMP) * EXP(1.E8/TEMP**3)
    K1 = F0 * EXP(G0/TEMP)
    K2 = C0 * K0/(K0+K1)
    ISO1 = A0 * EXP(B0/TEMP) * (-(K2-1))
    return ISO1
def EXP(x):
    return(math.exp(x))
def LOG10(x):
    return(math.log10(x))
def TUN(A0, B0, C0):
    return(A0 * EXP(-B0/TEMP) * EXP(C0/TEMP**3))
def ALK(A0, B0, C0, n, X0, Y0):
    K0 = 2.0E-22 * EXP(n)
    K1 = 4.3E-1 * (TEMP/298.0) ** (-8)
    K0 = K0 * CFACTOR
    K1 = K0/K1
    K2 = (K0/(1.0 + K1)) * 4.1E-1 ** (1.0/(1.0 + (LOG10(K1)) ** 2))
    K3 = C0/(K2 + C0)
    K4 = A0 * (X0 - TEMP * Y0)
    ALK = K4 * EXP(B0/TEMP) * K3
    return(ALK)
def NIT(A0, B0, C0, n, X0, Y0):
    K0 = 2.0E-22 * EXP(n)
    K1 = 4.3E-1 *(TEMP/298.0) ** (-8)
    K0 = K0 * CFACTOR
    K1 = K0/K1
    K2 = (K0/(1.0 + K1)) * 4.1E-1 ** (1.0 /(1.0 +(LOG10(K1)) ** 2))
    K3 = K2/(K2 + C0)
    K4 = A0*(X0 - TEMP * Y0)
    NIT = K4 * EXP(B0/TEMP) * K3
    return(NIT)
def ISO2(A0, B0, C0, D0, E0, F0, G0):
    K0 = D0 * EXP(E0/TEMP) * EXP(1.E8/TEMP**3)
    K1 = F0 * EXP(G0/TEMP)
    K2 = C0 * K0/(K0+K1)
    ISO2 = A0 * EXP(B0/TEMP) * K2
    return ISO2
def EPO(A1, E1, M1):
    K1 = 1 / (M1 * CFACTOR + 1)
    EPO = A1 * EXP(E1/TEMP) * K1
    return EPO
def KCO(A1, M1):
    KCO = A1 * (1 + (CFACTOR/M1))
    return KCO
def FALL(A0,B0,C0,A1,B1,C1,CF):
    K0 = A0 * EXP(B0/TEMP) * (TEMP/300)**C0
    K1 = A1 * EXP(B1/TEMP) * (TEMP/300)**(C1)
    K0 = K0*CFACTOR
    K1 = K0/K1
    FALL = (K0 / (1.00+K1) * CF**(1 / (1 + LOG10(K1)) **2))
    return FALL
def TROE(A0, B0, C0, A1, B1, C1, CF):
    K0 = A0 * EXP(B0/TEMP) * (TEMP/300) ** C0
    K1 = A1 * EXP(B1/TEMP) * (TEMP/300) ** C1
    K0 = K0 * CFACTOR
    KR = K0/K1
    NC = 0.75 - 1.27 * LOG10(CF)
    F = 10 ** ((LOG10(CF)) / (1+(LOG10(KR)/NC)**2))
    TROE = K0*K1*F / (K0+K1)
    return TROE
def ARR(A0, B0, C0):
    return(A0 * EXP(B0/TEMP) * EXP(C0/TEMP**3))
def K_OH_CO(T,M):
    T3I = 1/T
    KLO1=5.9e-33*(300*T3I)**(1.4)
    KHI1=1.1E-12*(300.*T3I)**(-1.3)
    XYRAT1=(KLO1*M)/KHI1
    BLOG1= np.log10(XYRAT1)
    FEXP1=1.0/(1.0+BLOG1*BLOG1)
    KCO1=KLO1*M*0.6**(FEXP1)/(1.0+XYRAT1)
    KLO2=1.5E-13*(300*T3I)**(-0.6)
    KHI2=2.1E9*(300*T3I)**(-6.1)
    XYRAT2 = KLO2*M/KHI2
    BLOG2=LOG10(XYRAT2)
    FEXP2=1.0/(1.0+ BLOG2*BLOG2)
    KCO2=KLO2*0.6**(FEXP2/(1.0+XYRAT2))
    KCO=KCO1+KCO2
    return KCO
def KRO2NO3():
    return 2.3e-12
def KAPHO2(T):
    return 5.2e-13*np.exp(980/T) 
def KAPNO(T):
    return 7.5e-12*np.exp(290/T)
def KNO3AL(T):
    return 1.44e-12*np.exp(-1862/T) ;
def KCH3O2(T):
    return 1.03e-13*np.exp(365/T)
def KBPAN(T,M):
    KD0 = 1.10e-05*M*np.exp(-10100/T)
    KDI = 1.90e17*np.exp(-14100/T)
    KRD = KD0/KDI
    FCD = 0.30
    NCD = 0.75-1.27*(np.log10(FCD)) ;
    FD = 10**(np.log10(FCD)/(1+(np.log10(KRD)/NCD)**2)) ;
    return (KD0*KDI)*FD/(KD0+KDI) ;
def KFPAN(T,M):
    KC0 = 3.28e-28*M*(T/300)**(-6.87) 
    KCI = 1.125e-11*(T/300)**(-1.105) 
    KRC = KC0/KCI 
    FCC = 0.30 
    NC = 0.75-1.27*(np.log10(FCC)) 
    FC = 10**(np.log10(FCC)/(1+(np.log10(KRC)/NC)**2)) ;
    return (KC0*KCI)*FC/(KC0+KCI) ;

#### Analysis Functions

In [5]:
#get_prod_reac(eq_list)
#takes eq list and separates into format reac_list2 = [[r1,r2],[r1],...], prod_list2 = [[r1,r2],[r1],...], 
#reac_coeff_list = [[1,1],[1],...], prod_coeff_list = [[1,1],[2],...] (as examples)
def get_prod_reac(eq_list):
    nums = ['0','1','2','3','4','5','6','7','8','9']
    reac_list = [i[0].split(' = ')[0].split(' + ') for i in eq_list]
    prod_list = [i[0].split(' = ')[1].split(' + ') for i in eq_list]
    prod_list2 = deepcopy(prod_list)
    prod_coeff_list = deepcopy(prod_list)
    reac_list2 = deepcopy(reac_list)
    reac_coeff_list = deepcopy(reac_list)
    a=[]
    for i in range(len(prod_list2)):
        for j in range(len(prod_list2[i])):
            if prod_list2[i][j][:1] in nums:
                a=re.split('([a-zA-Z])',prod_list[i][j],1)
                prod_list2[i][j]=a[1]+a[2]
                prod_coeff_list[i][j]=a[0]
            else:
                prod_coeff_list[i][j]=1
    b=[]
    for i in range(len(reac_list2)):
        for j in range(len(reac_list2[i])):
            if reac_list2[i][j][:1] in nums:
                b=re.split('([a-zA-Z])',reac_list[i][j],1)
                reac_list2[i][j]=b[1]+b[2]
                reac_coeff_list[i][j]=b[0]
            else:
                reac_coeff_list[i][j]=1
    for i in range(len(prod_coeff_list)):
        for j in range(len(prod_coeff_list[i])):
            prod_coeff_list[i][j] = float(prod_coeff_list[i][j])
    for i in range(len(reac_coeff_list)):
        for j in range(len(reac_coeff_list[i])):
            reac_coeff_list[i][j] = float(reac_coeff_list[i][j])
    return reac_list2,reac_coeff_list,prod_list2,prod_coeff_list

In [6]:
#get_network(species_list2,reac_list,prod_list,background_spc)
#creates reaction network of format network[reactant][product]
# in_list gives in_list[species] --> reactants that can react to form this species
# out_list gives in_list[species] --> products that are formed from this species
def get_network(species_list2,reac_list,prod_list,background_spc):
    network = np.zeros((len(species_list2),len(species_list2)))
    for i in range(len(reac_list)):
        for j in range(len(reac_list[i])):
            for k in range(len(prod_list[i])):
                if reac_list[i][j] not in  background_spc and prod_list[i][k] not in  background_spc:
                    network[species_list2.index(reac_list[i][j]),species_list2.index(prod_list[i][k])] = 1
    out_list = network_2_out_list(network,species_list2)
    in_list = network_2_in_list(network,species_list2)
    return network, in_list, out_list

In [7]:
#eqn_to_reactions(reac_list,reac_coeff_list,prod_list,prod_coeff_list,eq_list), with evaluated k
#creates list of reactions from eq_list and prod/reac lists
# format: [[[reactant,coeff],...], [[product,coeff],...],[rate constant,rate constant equation]]
def eqn_to_reactions(reac_list,reac_coeff_list,prod_list,prod_coeff_list,eq_list):
    reaction_list = []
    k_list = [eval(i[1]) for i in eq_list]
    for i in range(len(k_list)):
        rec = [[reac_list[i][j],-1*reac_coeff_list[i][j]] for j in range(len(reac_list[i]))]
        pro = [[prod_list[i][j],prod_coeff_list[i][j]] for j in range(len(prod_list[i]))]
        reaction_list.append([rec+pro,[k_list[i],eq_list[i][1]]])
    return reaction_list

In [8]:
#eqn_to_reactions2(reac_list,reac_coeff_list,prod_list,prod_coeff_list,eq_list), no evaluated k
#creates list of reactions from eq_list and prod/reac lists
# format: [[[reactant,coeff],...], [[product,coeff],...],rate constant equation]
def eqn_to_reactions2(reac_list,reac_coeff_list,prod_list,prod_coeff_list,eq_list):
    reaction_list = []
    for i in range(len(eq_list)):
        rec = [[reac_list[i][j],-1*reac_coeff_list[i][j]] for j in range(len(reac_list[i]))]
        pro = [[prod_list[i][j],prod_coeff_list[i][j]] for j in range(len(prod_list[i]))]
        reaction_list.append([rec+pro,eq_list[i][1]])
    return reaction_list

In [9]:
#network_to_reactions(network,out_list, species_list,reac_list, prod_list, background_spc, reaction_list)
#the confusing part addresses the situation in which a reaction is still there, but one of the products is 
#removed because the edge connected said product to the reactant is removed
def network_to_reactions(network,out_list, species_list,reac_list, prod_list, background_spc, reaction_list):
    species_list_new = get_species(network,species_list)
    species_list_new.extend(background_spc)
    reaction_list_new = []
    for i in range(len(reac_list)):
        reaction = reaction_list[i]
        if all(j in species_list_new for j in reac_list[i]):
            edge_list = []
            for m in reac_list[i]:
                for n in prod_list[i]:
                    edge_list.append(is_edge(m,n,out_list,species_list))
            if any(edge_list) or all(f in background_spc for f in prod_list[i]):
                for b in prod_list[i]:
                    if not any([is_edge(a,b,out_list,species_list) for a in reac_list[i]]):
                        for l in reaction[0]:
                            if l[0] is b and l[1]>0:
                                if l[0] not in background_spc:
                                    reaction[0].remove(l)
                reaction_list_new.append(reaction)
    return reaction_list_new

In [10]:
#out_list_2_network(out_list,species_list)
def out_list_2_network(out_list,species_list):
    network1 = np.zeros((len(species_list),len(species_list)))
    for i in range(len(out_list)):
        for j in range(len(out_list[i])):
            network1[i][out_list[i][j]] = 1
    return network1

In [11]:
#in_list_2_network(in_list,species_list)
def in_list_2_network(in_list,species_list):
    network1 = np.zeros((len(species_list),len(species_list)))
    for i in range(len(in_list)):
        for j in range(len(in_list[i])):
            network1[in_list[i][j]][i] = 1
    return network1

In [12]:
#network_2_out_list(network,species_list)
def network_2_out_list(network,species_list):
    out_list=[]
    for i in range(len(network)):
        a=[]
        for j in range(len(network)):
            if network[i][j] >0:
                a.append(j)
        out_list.append(a)     
    return out_list

In [13]:
#network_2_in_list(network,species_list)
def network_2_in_list(network,species_list):
    in_list=[]
    for i in range(len(network)):
        a=[]
        for j in range(len(network)):
            if network[j][i] >0:
                a.append(j)
        in_list.append(a)     
    return in_list

In [14]:
#out_list_2_in_list(out_list,species_list)
def out_list_2_in_list(out_list,species_list):
    network = out_list_2_network(out_list,species_list)
    a = []
    for i in range(len(network)):
        b =[]
        for j in range(len(network[i])):
            if network[j][i]==1:
                b.append(j)
        a.append(b)
                
    return a

In [15]:
#in_list_2_out_list(in_list,species_list)
def in_list_2_out_list(in_list,species_list):
    network = in_list_2_network(in_list,species_list)
    a = []
    for i in range(len(network)):
        b =[]
        for j in range(len(network[i])):
            if network[j][i]==1:
                b.append(j)
        a.append(b)
                
    return a

In [16]:
#reaction_list_2_eq_list(reaction_list), includes rate
def reaction_list_2_eq_list(reaction_list):
    eq_list = []
    for i in reaction_list:
        length = len(i[0])
        eq_string = ''
        coeff = -1
        counter = 0
        while coeff < 0:
            eq_string = eq_string + str(-1*i[0][counter][0])+i[0][counter][0]
            if counter < length-1:
                coeff = i[0][counter + 1][1]
            else:
                break
            if coeff < 0:
                eq_string = eq_string + ' + '
            else:
                eq_string = eq_string + ' = '
            counter = counter + 1
        while counter < len(i[0]):
            eq_string = eq_string + str(-1*i[0][counter][0])+i[0][counter][0]
            counter = counter + 1
            if counter < len(i[0]):
                eq_string = eq_string + ' + '
        eq_list.append([eq_string,i[1]])
    return eq_list

In [17]:
#eqn_to_reactions(reac_list,reac_coeff_list,prod_list,prod_coeff_list,eq_list), accounts for j rates
#creates list of reactions from eq_list and prod/reac lists
# format: [[[reactant,coeff],...], [[product,coeff],...],[rate constant,rate constant equation]]
def eqn_to_reactions(reac_list,reac_coeff_list,prod_list,prod_coeff_list,eq_list):
    reaction_list = []
    for i in eq_list:
        if i[1] == 'J(22) ':
            I = 5.804e-6;
            m = 1.092;
            n = 0.377;
            k_list.append(j_func(sza,I,m,n))
        elif i[1] == 'J(34) ':
            I = 1.537e-4;
            m = 0.170;
            n = 0.208;
            k_list.append(j_func(sza,I,m,n))
        elif i[1] == 'J(41) ':
            I = 7.649e-6;
            m = 0.682;
            n = 0.279;
            k_list.append(j_func(sza,I,m,n))
        else:
            k_list.append(eval(i[1])) 
    for i in range(len(k_list)):
        rec = [[reac_list[i][j],-1*reac_coeff_list[i][j]] for j in range(len(reac_list[i]))]
        pro = [[prod_list[i][j],prod_coeff_list[i][j]] for j in range(len(prod_list[i]))]
        reaction_list.append([rec+pro,[k_list[i],eq_list[i][1]]])
    return reaction_list

In [18]:
#pressure_to_m(P,T)
#number density is molecules per cubic centimeter
#it is n = p/kT, if p is in mbar, then that is equivalent to 100 pascals, so we multiple k by 
def pressure_to_m(P,T):
    Na = 6.022e23; #molecules per mole
    R = 8.314e4; # cm^3 mbar /K /mol
    M = Na*P/(R*T)
    return M

In [19]:
#j_func(sza,I,m,n)
#J = I * cos(SZA)^m * exp(-n * sec(SZA))
def j_func(sza,I,m,n):
    a = I*(np.cos(sza*(2*np.pi)/360)**m)
    b = np.exp(-n/(np.cos(sza*(2*np.pi)/360)))
    return a*b

In [20]:
#network_to_reactions(network,out_list, species_list,reac_list, prod_list, background_spc, reaction_list)
def network_to_reactions(network,out_list, species_list,reac_list, prod_list, background_spc, reaction_list):
    species_list_new = get_species(network,species_list)
    species_list_new.extend(background_spc)
    reaction_list_new = []
    for i in range(len(reac_list)):
        reaction = reaction_list[i]
        if all(j in species_list_new for j in reac_list[i]):
            edge_list = []
            for m in reac_list[i]:
                for n in prod_list[i]:
                    edge_list.append(is_edge(m,n,out_list,species_list))
            if any(edge_list) or all(f in background_spc for f in prod_list[i]):
                for b in prod_list[i]:
                    if not any([is_edge(a,b,out_list,species_list) for a in reac_list[i]]):
                        for l in reaction[0]:
                            if l[0] is b and l[1]>0:
                                if l[0] not in background_spc:
                                    reaction[0].remove(l)
                reaction_list_new.append(reaction)
    return reaction_list_new

In [21]:
#edge_to_reactions(edge,reac_list,reac_coeff_list,prod_list,prod_coeff_list,k_list)
def edge_to_reactions(edge,reac_list,reac_coeff_list,prod_list,prod_coeff_list,k_list):
    reactions = []
    for i in range(len(reac_list)):
        if species_list[edge[0]] in reac_list[i] and species_list[edge[1]] in prod_list[i]:
            reac_str = ''
            reac = []
            prod = []
            rate = [k_list[i]]
            for j in range(len(reac_list[i])):
                reac_str = reac_str + str(reac_coeff_list[i][j]) + reac_list[i][j] + ' + '
                reac.append([reac_list[i][j],reac_coeff_list[i][j]])
                if reac_list[i][j] != species_list[edge[0]]:
                    rate.append(reac_list[i][j])
            reac_str = reac_str + '='
            for k in range(len(prod_list[i])):
                reac_str = reac_str + ' + ' + str(prod_coeff_list[i][k]) + prod_list[i][k]
                prod.append([prod_list[i][k],prod_coeff_list[i][k]])
            reactions.append([reac_str,reac,prod,rate])
    return reactions

In [22]:
#get_sequences(species_list,out_list,in_list)
def get_sequences(species_list,out_list,in_list):
    sequences = []
    for i in range(len(species_list)):
        if len(out_list[i]) == 1 and len(in_list[i]) == 1:
            check = 1
            seq = [i]
            j = i
            b = i
            while check > 0:
                if len(out_list[out_list[j][0]]) == 1 and len(in_list[out_list[j][0]]) == 1:
                    seq.append(out_list[j][0])
                    j = out_list[j][0]
                    check1 = 1
                else:
                    check1 = 0
                if seq not in sequences:
                    sequences.append(seq)
                if len(in_list[in_list[b][0]]) == 1 and len(out_list[in_list[b][0]]) == 1:
                    seq.insert(0,in_list[b][0])
                    b = in_list[b][0]
                    check2 = 1
                else:
                    check2 = 0
                if check2 == 1 or check1 == 1:
                    check = 1
                else:
                    check = 0
            if seq not in sequences:
                sequences.append(seq)
    sequences2 = []
    for i in sequences:
        if i not in sequences2:
            sequences2.append(i)
    return sequences2

In [23]:
#reduce_sequences(species_list, net, out_list, in_list)
def reduce_sequences(species_list, net, out_list, in_list):
    sequences = get_sequences(species_list,out_list,in_list)
    actions = []
    net_working = deepcopy(net)
    in_list_working = deepcopy(in_list)
    out_list_working = deepcopy(out_list)
    for i in sequences:
        giver = in_list[i[0]][0]
        reciever = out_list_working[i[-1]][0]
        if reciever not in out_list_working[giver]:
            out_list_working[giver].append(reciever)
            net_working[giver][reciever] = 1
            in_list_working[reciever].append(giver)
        action_node = []
        action_edge = []
        for j in i:
            action_node.append(j)
            if i.index(j) == 0:
                action_edge.append([in_list_working[j][0],j])
                action_edge.append([j,out_list_working[j][0]])
            else:
                action_edge.append([j,out_list_working[j][0]])
            del out_list_working[j][:]
            del in_list_working[j][:]
            for a in range(len(net_working)):
                for b in range(len(net_working[i])):
                                    
                    if a == j or b == j:
                        net_working[a][b] == 0
        actions.append([action_node,action_edge])
    return actions, out_list_working, in_list_working, net_working

In [24]:
#get_super_nodes(species_list, net, out_list, in_list)
def get_super_nodes(species_list, net, out_list, in_list):
    matches = []
    for i in range(len(species_list)):
        for j in range(len(species_list)):
            if i != j:
                if out_list[i] == out_list[j]:
                    matches.append([i,j])
    super_nodes = []
    counted = []
    for i in matches:
        if i[0] not in counted and i[1] not in counted:
            counted.append(i[0])
            counted.append(i[1])
            super_nodes.append(i)
        elif i[0] not in counted and i[1] in counted:
            counted.append(i[0])
            for k in range(len(super_nodes)):
                if i[1] in super_nodes[k]:
                    super_nodes[k].append(i[0])
        elif i[1] not in counted and i[0] in counted:
            counted.append(i[1])
            for k in range(len(super_nodes)):
                if i[0] in super_nodes[k]:
                    super_nodes[k].append(i[1])
    return super_nodes

In [25]:
#reduce_super_nodes(species_list,net,out_list,in_list)
def reduce_super_nodes(species_list,net,out_list,in_list):
    super_nodes = get_super_nodes(species_list,net,out_list,in_list)
    actions = []
    out_list_working = deepcopy(out_list)
    in_list_working = deepcopy(in_list)
    net_working = deepcopy(net)
    for i in super_nodes:
        for j in range(len(i)-1):
            actions.append([i[j+1],i[0]])
            for k in in_list_working[i[j+1]]:
                if k not in in_list_working[i[0]]:
                    in_list_working[i[0]].append(k)
                    out_list_working[k].append(i[0])
                #actions.append([[k,i[j+1]],[k,i[0]]])
            del out_list_working[i[j+1]][:]
            del in_list_working[i[j+1]][:]
            for a in range(len(net_working)):
                for b in range(len(net_working[a])):
                    if a == i[j+1] or b == i[j+1]:
                        net_working[a][b] == 0
    return actions, out_list_working, in_list_working, net_working

In [26]:
#create_f0am_file(network,reaction_list,species_list,name)
def create_f0am_file(network,reaction_list,species_list,name):
    spec_list2  = get_species(network,species_list)
    spec_2_add = "SpeciesToAdd = {'CH3OO';'CH3NO3';'CH3O';'CH3OOH';'IDNOO';'IHNEOO';'NO3CH2PAN';'NPAH';'NO3CH2CO2H';'NCH2CO3';'NCH2CO2H';'NCH2CO3H';'PNAN';'IDHNBOO';'MPANHN';'MACRNO2';'PAN';'CH3CO3H';'CH3CO2H';'MACRNOOH';'MACRNOH';'PHAN';'HOCH2CO3H';'HOCH2CO2H';'HOCH2COCHO';'HOCH2CO3';'HCOCO';'MO2';'OH';'CH3OOH';'N2O5';'HNO3';'CH3NO3';'SO2';'O1D';'H2'; 'O';'SO3';'CH3OONO2';'CH3O';'HONO';'SA';'HSO3';'NA';'HO2NO2';'ISOP';'ISOP1OHt';'ISOP1OHc';'ISOP4OHt';'ISOP4OHc';'ISOP1OH2OO';'ISOP1OH4OOt';'ISOP1OH4OOc';'ISOP1OO4OHt';'ISOP1OO4OHc';'ISOP3OO4OH';'ISOP1OH2OOH';'ISOP3OOH4OH';'MVK';'MACR';'ISOP1CO4OH';'ISOP1OH4CO';'ISOP1OH2N';'ISOP1OH4Nc';'ISOP1OH4Nt';'ISOP3N4OH';'ISOP1N4OHc';'ISOP1N4OHt';'ISOP1CO2OOH3OO4OH';'ISOP1CO2OO3OOH4OH';'ISOP1CO2N3OOH4OH';'ISOP1CO2OOH3N4OH';'MVK3OOH4OH';'MGLY';'GLYC';'HCHO';'GLYX';'HAC';'ISOP1OH2OOH3OO4CO';'ISOP1OH2OO3OOH4CO';'ISOP1OH2OOH3N4CO';'ISOP1OH2N3OOH4CO';'ISOP1OH2OOH3OOH4CO';'ISOP1CO2OOH3OOH4OH';'MACR2OOH3OH';'MVK3OH4OH';'MACR3CO';'MACR2OH3OH';'ISOP3CO4OH';'ISOP1OH2OO3CO4OH';'MVK3CO4OH';'HPA';'ISOP1OH2N3CO4OH';'ISOP3OH4OH';'ISOP1OH2OH';'ISOP1OH2OOH3CO4OH';'ISOP1OH4OH';'CH3OO';'CH3OH';'ISOP1OOH4OHc';'ISOP1OH4OOHc';'ISOP1OOH4OHt';'ISOP1OH4OOHt';'ISOP1CO4OOHc';'ISOP1CO2R3OOH4OOH';'ISOP1CO2OO3OOH4OOH';'ISOP1CO23O4OOH';'ISOP1CO2OOH3OO4OOH';'ISOP1CO2OOH3OOH4OO';'MVK3OOH4OOH';'MVK3OOH4CO4OOH';'MACR2OOH3CO3OOH';'ISOP1CO2N3OOH4OOH';'HPETHNL';'ISOP1CO2OOH3N4OOH';'ISOP1CO2OOH3OOH4OOH';'ISOP1CO2OOH3OOH4N';'MACR2OOH3CO';'HCOOH';'ISOP1OOH4COc';'ISOP1OOH2OOH3R4CO';'ISOP1OOH2OOH3OO4CO';'ISOP1OOH23O4CO';'ISOP1OOH2OO3OOH4CO';'ISOP1OO2OOH3OOH4CO';'MACR2OOH3OOH';'ISOP1OOH2OOH3N4CO';'ISOP1OOH2OOH3OOH4CO';'ISOP1OOH2N3OOH4CO';'ISOP1N2OOH3OOH4CO';'ISOP1OOH23O3OH4N';'ISOP1N2OOH34O4OH';'ISOP1OH12O3OOH4N';'ISOP1CO2OOH3OOH4CO';'ISOP1OOH2OOH3CO4CO';'ISOP1CO2OOH34O4OH';'ISOP1OH12O3OOH4CO';'HPAC';'MVK3OOH4CO';'MVK3OO4OH';'MVK3OH4OO';'CH3CO3';'MVK3OOH4OH4OH';'MVK3OH4OOH';'MVK3N4OH';'MVK3N4OH4OH';'MVK3OH4N';'ETHLN';'MACR2OO3OH';'MACR2OH3OO';'MACR1OO';'MACR2OH3OOH';'MACR1OOH';'CH3COOCH2';'MACR1OH';'MACR2N3OH';'MACR2OH3N';'MPAN';'MPAN1OHx';'MPAN1OH2OO';'HMMLx';'MPAN1OH';'HMML';'MPAN1OH2O';'MPAN1OH2OOH';'ISOP1OH2OOH3R4OH';'ISOP1OH2OOH3OO4OH';'ISOP1OH23O4OHt';'ISOP1OH23O4OHc';'ISOP1OH2R3OOH4OH';'ISOP1OH2OO3OOH4OH';'ISOP12O3OH4OH';'ISOP1OH2OOH3OH4CO';'ISOP1OH2OOH3OH4OO';'ISOP1OH2OO3OH4OOH';'ISOP1OH2OOH3OH4N';'ISOP1OH2N3OH4OOH';'ISOP1OH2OOH3OH4OOH';'ISOP1OO2OH3OOH4OH';'ISOP1OOH2OH3CO4OH';'ISOP1OOH2OH34O4OH';'ISOP1OOH2OH3OO4OH';'ISOP1N2OH3OOH4OH';'ISOP1OOH2OH3N4OH';'ISOP1OOH2OH3OOH4OH';'ISOP1CO2OH3OOH4OH';'ISOP1OH2OOH3N4OH';'ISOP1OH2OOH3OOH4OH';'ISOP1OH12O3OOH4OH';'ISOP1OH12O3OH4OOH';'ISOP1OH2N3OOH4OH';'ISOP1OH2OOH34O4OH';'ISOP1N2OOH3OH4CO';'ISOP1CO2OH3OOH4N';'ISOP1OOH2OH3CO4N';'ISOP1N4R4OH';'ISOP1OH1R4N';'ISOP3OH3R4N';'ISOP1OOH2R3CO4N';'ISOP1N2OOH3OO4CO';'ISOP1N2OO3OOH4CO';'ISOP1CO2OO3OOH4N';'ISOP1CO2OOH3OO4N';'ISOP1CO2OOH';'ISO1OH12O';'ISOP3OOH4CO';'ISOP1OH23O4R4OHc';'ISOP1OH23O4CO';'ISOP1OH2OH3OO4CO';'ISOP1OH1R23O4OHc';'ISOP1CO23O4OH';'ISOP1CO2OO3OH4OH';'ISOP1OH23O4R4OHt';'ISOP1OH1R23O4OHt';'ISOP1OH2OH34O';'ISOP1CO2OH34O';'ISOP12O3OH4CO';'ISOP12O3OH3R4OH';'ISOP12O3CO4OH';'ISOP1OH3OH4CO';'ISOP1CO3OH4OH';'ISOP1OH2OH3N4CO';'ISOP1OH2OH3OOH4CO';'ISOP1CO2OOH3OH4OH';'ISOP1CO2N3OH4OH';'MVK3OH4CO';'MACR2OH3CO';'MVKENOL';'C4HVP1';'ISOP1CO4CO';'ISOP1CO2OO3OOH4CO';'MACRENOL';'C4HVP2';'ISOP1CO2OOH3OO4CO';'MVK3OO4OH4OH';'DHA';'PPYRAC';'PYRAC';'ciHCHOO';'ciMVKOO';'ciMACROO';'H2SO4';'HPMF';'HMHP';'H2O2';'MACR3OH3OOH';'MVK3OH3OOH';'FAH';'ISOP1CO4OOc';'ISOP1CO2OH3R4OOH';'ISOP1CO2OO3OH4OOH';'ISOP1CO2OOH3OH4CO';'ISOP1CO2OH3OO4OOH';'ISOP1OH2OOH3CO4CO';'ISOP1CO3R4OOH';'ISOP1CO2OH3OOH4OO';'ISOP1CO2OOH3OH4OO';'ISOP1CO2OO';'ISOP1OO4COc';'ISOP1OOH2OH3OO4CO';'ISOP1CO2OH3OOH4CO';'ISOP1OOH2OH3CO4CO';'ISOP1OOH2R3OH4CO';'ISOP1OOH2OO3OH4CO';'ISOP1OOH2R4CO';'ISOP1OO2OH3OOH4CO';'ISOP1OO2OOH3OH4CO';'ISOP3OO4CO';'ISOP1OH2N3R4OH';'ISOP1OH2N3OO4OH';'ISOP1OH2N3OH4OO';'ISOP1OH2N3OH4N';'ISOP1OH2N3N4OH';'ISOP1CO2N3OH4OOH';'ISOP1OH2R3N4OH';'ISOP1OH2OO3N4OH';'ISOP1OO2OH3N4OH';'ISOP1N2OH3N4OH';'ISOP1OOH2OH3N4CO';'ISOP1N2OO';'ISOP3OO4N';'ISOP1N4OO';'ISOP1OO4N';'ISOP1N2OOH';'ISOP1N4OOH';'ISOP3OOH4N';'ISOP1OOH4N';'ISOP1N2OOISOP1N2';'ISOP1N2OH';'ISOP3OO4NISOP1N2';'ISOP3CO4N';'ISOP1CO4N';'ISOP1O4N';'ISOP1OO4NISOP1N2';'ISOP1N4CO';'ISOP1N4O';'ISOP1N2OOISOP1N4';'ISOP1N4OOISOP1N4';'ISOP3OH4N';'ISOP3OO4NISOP1N4';'ISOP1OO4NISOP1N4';'ISOP1N2N';'ISOP1N4N';'ISOP3N4N';'ISOP1N253OO4OH';'ISOP1N253O4OH';'ISOP1N253N4OH';'MACR4N';'ISOP1N253OOH4OH';'ISOP1N253CO4OH';'ISOP1N253OH4OH';'ISOP1N253OO4OHISOP1N4';'ISOP1N253OO4OHISOP1N2';'ISOP1N2OOH3R4OH';'ISOP1N2OOH3OO4OH';'ISOP1N23O4OH';'ISOP1N2OOH3OH4OO';'ISOP1N2OH3R4OOH';'ISOP1N2OH3OO4OOH';'ISOP1N2OH34O';'ISOP1N2R3OH4OOH';'ISOP1N2OO3OH4OOH';'ISOP12O3OH4OOH';'ISOP1OH2R3OOH4N';'ISOP1OH2OO3OOH4N';'ISOP1OH23O4N';'ISOP1OO2OH3OOH4N';'ISOP1OOH2R3OH4N';'ISOP1OOH2OO3OH4N';'ISOP12O3OH4N';'ISOP1OOH2OH3R4N';'ISOP1OOH2OH3OO4N';'ISOP1OOH2OH34O';'ISOP1N2OOH3N4OH';'PROPNN';'MACR2OOH3N';'ISOP1N2OOH3OOH4OH';'ISOP1N2OOH3OH4N';'ISOP1N2OOH3OH4OOH';'ISOP1N2OH3N4OOH';'ISOP1N2OH3OOH4OOH';'ISOP1N2N3OH4OOH';'ISOP1OH2N3OOH4N';'MVK3OOH4N';'ISOP1OH2OOH3OOH4N';'ISOP1N2OH3OOH4N';'ISOP1OOH2OH3OOH4N';'ISOP1OOH2N3OH4N';'ISOP1OOH2OOH3OH4N';'ISOP1OOH2OH3N4N';'ISOP1N2OO3OOH4OH';'ISOP1N2OH3OOH4OO';'ISOP1OH2OOH3OO4N';'ISOP1OO2OOH3OH4N';'ISOP1N2N3OOH4OH';'ISOP1OH2OOH3N4N';'ISOP1N23O4OH4R';'ISOP1N23O4CO';'ISOP1N2OH3OO4CO';'ISOP1OH1R23O4N';'ISOP1CO23O4N';'ISOP1CO2OO3OH4N';'ISOP1N2OH3CO4OO';'ISOP1N2OH3CO4OOH';'ISOP12O3OH3R4N';'ISOP12O3CO4N';'ISOP1OH2OO3CO4N';'ISOP1N2OH3CO4N';'ISOP1OH2OOH3CO4N';'NPA';'MVK3CO4N';'ISOP1OH2N3CO4N';'ISOP1N2OO3CO4OH';'ISOP1N2N3CO4OH';'ISOP1N4R4CO';'ISOP1N4CO4OO';'ISOP1N4PAN';'C4NVP2';'ISOP1N4CO4OOH';'ISOP1N4CO4OH';'MACR2OO3N';'MACR2N3N';'ISOP1CO1R4N';'ISOP1CO1OO4N';'ISOP1PAN4N';'C4NVP1';'ISOP1CO1OOH4N';'ISOP1CO1OH4N';'MVK3OO4N';'MVK3N4N';'ISOP1N2R3OH4CO';'ISOP1N2OO3OH4CO';'ISOP1CO2OH3R4N';'ISOP1CO2OH3OO4N';'ISOP1N2OH3OOH4CO';'ISOP1N2OH3N4CO';'ISOP1CO2OOH3OH4N';'ISOP1CO2N3OH4N';'ISOP1N2OH3OO4OH';'ISOP1N2OH3OH4OO';'ISOP1N2R3OH4OH';'ISOP1N2OO3OH4OH';'ISOP1OH2OO3OH4N';'ISOP1OO2OH3OH4N';'ISOP1OH2OH3R4N';'ISOP1OH2OH3OO4N';'ISOP1N2OOH3OH4OH';'ISOP1OH2OH3OOH4N';'ISOP1N2N3OH4OH';'ISOP1OH2OH3N4N';'ISOP1N2OH3OH4OOH';'ISOP1OOH2OH3OH4N';'ISOP1N2OH3OH4N';'ISOP1CO1OO4OOH';'ISOP1CO1OOH4OO';'ISOP1CO1OOH2OO';'ISOP1CO1OOH';'ISOP4CO4OOH';'ISOP1OOH4CO4OO';'ISOP1OO4CO4OOH';'ISOP3OO4CO4OOH';'ISOP1CO1R4OH';'ISOP1OH4CO4R';'ISOP1CO1OO4OH';'ISOP1OH4CO4OO';'ISOP1PAN4OH';'ISOP1OH4PAN';'ISOP1CO1OH4OH';'ISOP1OH4CO4OH';'ISOP1CO1OOH4OH';'ISOP1OH4CO4OOH';'CO';'CO2';'M';'O2';'OH';'HO2';'NO';'NO2';'NO3';'H2O';'H2Od';'O3';'SO2';'CH3OH'};"
    '''for i in node_list:
        spec_2_add = spec_2_add + "'" + i +"'"+ ';'
    spec_2_add = spec_2_add[:-1]   
    spec_2_add = spec_2_add  + "};"'''
    #adds equations to string
    eq_str_a = ''
    for i in range(len(eq_list_addition)):
        eq_str_a = eq_str_a + "\ni=i+1;\nRnames{i} = '" + eq_list_addition[i][0] + "';\nk(:,i) = "+eq_list_addition[i][1] + ';\n'
        for j in range(len(reac_list2_a[i])):
            if reac_list2_a[i][j]!='hv':
                eq_str_a = eq_str_a + 'Gstr{i,'+str(j+1)+"} = '"+reac_list2_a[i][j]+"'; "
        eq_str_a = eq_str_a +'\n'
        for j in range(len(reac_list2_a[i])):
            if reac_list2_a[i][j]!='hv':
                eq_str_a = eq_str_a + 'f'+ reac_list2_a[i][j]+'(i)'+'='+'f'+ reac_list2_a[i][j]+'(i)'+str(-1*reac_coeff_list_a[i][j]) + '; '
        for j in range(len(prod_list2_a[i])):
            eq_str_a = eq_str_a + 'f'+ prod_list2_a[i][j]+'(i)'+'='+'f'+ prod_list2_a[i][j]+'(i)'+'+'+str(prod_coeff_list_a[i][j]) + '; '
        eq_str_a = eq_str_a +'\n'
        eq_str_a = eq_str_a +'\n'
    eq_str = ''
    eq_list_r = reaction_list_2_eq_list(reaction_list)
    for i in range(len(reaction_list)):
        eq_str = eq_str + "\ni=i+1;\nRnames{i} = '" + eq_list_r[i][0] + "';\nk(:,i) = "+reaction_list[i][1] + ';\n'
        counter = 1
        for j in reaction_list[i][0]:
            if j[1]<0:
                eq_str = eq_str + 'Gstr{i,'+str(counter)+"} = '"+j[0]+"'; "
                counter = counter + 1
        eq_str = eq_str +'\n'
        for k in reaction_list[i][0]:
            if k[1]<0:
                eq_str = eq_str + 'f'+ k[0]+'(i)'+'='+'f'+ k[0]+'(i)'+str(k[1]) + '; '
            else:
                eq_str = eq_str + 'f'+ k[0]+'(i)'+'='+'f'+ k[0]+'(i)'+'+'+str(k[1]) + '; '
        
        eq_str = eq_str +'\n'
        eq_str = eq_str +'\n'
    eq_str = eq_str.replace('CH2O','HCHO')
    eq_str = eq_str.replace('.*RO2',';%.*RO2')
    eq_str = eq_str.replace('J(22)','J22')
    eq_str = eq_str.replace('J(34)','J34')
    eq_str = eq_str.replace('J(41)','J41')
    eq_str = eq_str.replace('ALK(','F0AM_isop_ALK(T,M,')
    eq_str = eq_str.replace('EPO(','F0AM_isop_EPO(T,M,')
    eq_str = eq_str.replace('NIT(','F0AM_isop_NIT(T,M,')
    eq_str = eq_str.replace('TROE(','F0AM_isop_TROE2(T,M,')
    eq_str = eq_str.replace('TUN(','F0AM_isop_TUN(T,M,')
    eq_str = eq_str.replace('ISO1(','F0AM_isop_ISO1(T,')
    eq_str = eq_str.replace('ISO2(','F0AM_isop_ISO2(T,')
    eq_str = eq_str.replace('KCO(','F0AM_isop_KCO(T,M,')
    eq_str = eq_str.replace('FALL(','F0AM_isop_FALL(T,M,')
    eq_str = eq_str.replace('TEMP','T')
    eq_str = eq_str.replace('**','.^')
    eq_str = eq_str.replace('EXP','exp')
    eq_str = eq_str.replace('*','.*')
    eq_str = eq_str.replace('KAPHO2(T)','KAPHO2')
    eq_str = eq_str.replace('KFPAN(T,M)','KFPAN')
    eq_str = eq_str.replace('KNO3AL(T)','KNO3AL')
    eq_str = eq_str.replace('KAPNO(T)','KAPNO')
    eq_str = eq_str.replace('KBPAN(T,M)','KBPAN')
    eq_str = eq_str.replace('KRO2NO3()','KRO2NO3')
    full = spec_2_add + '\n'+'RO2ToAdd = {};'+'\n'+'AddSpecies'+'\n'+ eq_str_a + eq_str
    f0am_file = open("f0am_"+name+".txt","w+")
    f0am_file.write(full)

In [27]:
#just_plot(species_list1,eq_list1,background_spc,name)
def just_plot(species_list1,eq_list1,background_spc,name):
    reac_list,reac_coeff_list,prod_list,prod_coeff_list = get_prod_reac(eq_list1)
    conc_max = np.zeros(len(species_list1))
    reaction_list = eqn_to_reactions(reac_list,reac_coeff_list,prod_list,prod_coeff_list,eq_list1)
    network, in_list, out_list = get_network(species_list1,reac_list,prod_list,background_spc)
    plot_mechanism(out_list,species_list1,name)

In [28]:
#plot_mechanism(net,species_list,name)
def plot_mechanism(net,species_list,name):
    species_list_new = get_species(net,species_list)
    G=pgv.AGraph(directed=True)
    for i in range(len(species_list_new)):
        G.add_node(species_list_new[i],label=species_list_new[i])
    for i in range(len(net)):
        for j in range(len(net[i])):
            if net[i][j]>0:
                G.add_edge(species_list[i],species_list[j])
    G.layout(prog="dot")
    G.draw(name + '.png')

In [29]:
#reduce_nodes(species_list, net, out_list, in_list)
def reduce_nodes(species_list, net, out_list, in_list):
    matches = []
    for i in range(len(species_list)):
        for j in range(len(species_list)):
            if out_list[i] == out_list[j]:
                matches.append([i,j])
    super_nodes = []
    counted = []
    for i in matches:
        if i[0] not in counted and i[1] not in counted:
            counted.append(i[0])
            counted.append(i[1])
            super_nodes.append(i)
        elif i[0] not in counted and i[1] in counted:
            counted.append(i[0])
            for k in range(len(super_nodes)):
                if i[1] in super_nodes[k]:
                    super_nodes[k].append(a)

In [30]:
#get_drg2(species_list,reac_list,prod_list,background_spc,rates,method)
def get_drg2(species_list,reac_list,prod_list,background_spc,rates,method):
    matrix_p = np.zeros((len(species_list),len(species_list)))
    matrix_c = np.zeros((len(species_list),len(species_list)))
    for i in range(len(prod_list)):
        for j in range(len(prod_list[i])):
            for k in range(len(reac_list[i])):
                if prod_list[i][j] not in  background_spc and reac_list[i][k] not in  background_spc:
                    matrix_p[int(species_list.index(prod_list[i][j])),int(species_list.index(reac_list[i][k]))] = matrix_p[int(species_list.index(prod_list[i][j])),int(species_list.index(reac_list[i][k]))]+ rates[i]
    for i in range(len(reac_list)):
        for j in range(len(reac_list[i])):
            for k in range(len(prod_list[i])):
                if reac_list[i][j] not in  background_spc and prod_list[i][k] not in  background_spc:
                    matrix_c[int(species_list.index(reac_list[i][j])),int(species_list.index(prod_list[i][k]))] = matrix_c[int(species_list.index(reac_list[i][j])),int(species_list.index(prod_list[i][k]))]+ rates[i]
    x_a = [sum(matrix_c[i,:]) for i in range(len(species_list))]
    p_a = [sum(matrix_p[i,:]) for i in range(len(species_list))]
    p_ab = deepcopy(matrix_p)
    x_ab = deepcopy(matrix_c)
    rp_ab = deepcopy(p_ab)
    for i in range(len(species_list)):
        for j in range(len(species_list)):
            if p_a[i] != 0:
                rp_ab[i][j] = p_ab[i][j]/p_a[i]
            else:
                rp_ab[i][j]           
    rm_ab = deepcopy(x_ab)
    for i in range(len(species_list)):
        for j in range(len(species_list)):
            if x_a[i] != 0:
                rm_ab[i][j] = x_ab[i][j]/x_a[i]
            else:
                rm_ab[i][j]
    if method == 'produced':
        drg_mat = np.transpose(rp_ab)
    # lu and law
    elif method == 'consumed':
        drg_mat = rm_ab
    elif method == 'sum':
        drg_mat = rm_ab + np.transpose(rp_ab)
    order_list = []
    for i in range(len(drg_mat)):
        for j in range(len(drg_mat[i])):
            if drg_mat[i][j]>0:
                order_list.append([drg_mat[i][j],i,j])
    order_list = sorted(order_list, key=lambda x: x[0])
    return drg_mat

In [31]:
#is_last(species,out_list)
def is_last(species,out_list):
    if len(out_list[species]) == 0:
        return True
    else:
        return False

In [32]:
#is_first(species,in_list)
def is_first(species,in_list):
    if len(in_list[species])==0:
        return True
    else:
        return False

In [33]:
#is_similar(species_A,species_B,network)
def is_similar(species_A,species_B,network):
    nt = np.transpose(network)
    if all(map(lambda x, y: x == y, network[i], network[j])):
        if all(map(lambda x, y: x == y, nt[i], nt[j])):
            return True
    else:
        return False

In [34]:
#get_rand_path(network,species)
def get_rand_path(network,species):
    ide = species
    nt = np.transpose(network)
    check = nt[ide]
    path = [species]
    while sum(nt[ide]) > 0:
        list_check = []
        for i in range(len(nt[ide])):
            if nt[ide][i]>0:
                list_check.append(i)
        if 0 in list_check:
            ide = 0
        else:
            ide = random.choice(list_check)
        path.append(ide)
    return path

In [35]:
#get_oxidants(path,networ)
def get_oxidants(path,networ):
    oxidants = []
    for i in range(len(path)-1):
        oxidants.extend(networ[path[i+1]][path[i]])
    return oxidants

In [36]:
#get_oxidation(species,in_list,out_list,network)
def get_oxidation(species,in_list,out_list,network):
    if is_last(species,out_list):
        oxidation = 1
    elif is_first(species, in_list):
        oxidation = 0
    else:
        paths_to = []
        paths_from = []
        for i in range(100):
            paths_to.append(len(clean_path(get_rand_path(network,species),in_list)))
            paths_from.append(len(clean_path(get_rand_path_from(network,species,out_list),in_list)))
        oxidation = sum(paths_from)/(sum(paths_to) + sum(paths_from))
    return oxidation

In [37]:
#clean_path(path,in_list)
def clean_path(path,in_list):
    new_path = list(dict.fromkeys(path))
    for j in range(6):
        for i in range(len(new_path)-1):
            if new_path[i+1] not in in_list[new_path[i]]:
                new_path_new = new_path[0:i]
                new_path_new.extend(new_path[i+1:len(new_path)])
                break
            else:
                new_path_new = deepcopy(new_path)
        new_path = deepcopy(new_path_new)
    return new_path

In [38]:
#get_rand_path_from(network,species,out_list)
def get_rand_path_from(network,species,out_list):
    ide = species
    nt = np.transpose(network)
    check = nt[ide]
    path = [species]
    while sum(network[ide]) > 0:
        list_check = []
        for i in range(len(network[ide])):
            if network[ide][i]>0:
                list_check.append(i)
        ide = random.choice(list_check)
        path.append(ide)
    return path

In [39]:
#get_set_distance(sample_start,samples)
def get_set_distance(sample_start,samples):
    sample_set = [samples[i] for i in sample_start]
    sample_set2 = np.transpose(sample_set)   
    l = len(sample_set2)
    dev = []
    for i in sample_set2:
        m = np.abs(np.mean(i))
        s = np.std(i)
    dev.append(s/m)
    return np.mean(dev)/l

In [40]:
#wide_set(tries,start,samples)
def wide_set(tries,start,samples):
    #get_set_distance(start)
    for i in range(tries):
        r = random.randint(1,100)
        if r not in start:
            for i in range(len(start)):
                startnew = deepcopy(start)
                startnew.remove(startnew[i])
                startnew.append(r)
                if get_set_distance(startnew,samples)>get_set_distance(start,samples):
                    start = deepcopy(startnew)
                    break
    return start

In [41]:
#list_diff(l1,l2)
def list_diff(l1,l2):
    a = []
    for i in range(len(l1)):
        if i in l1 and i in l2:
            a.append(np.abs(l1.index(i)-l2.index(i)))
    return np.mean(a)

In [42]:
#list_diff_sq(l1,l2)
def list_diff_sq(l1,l2):
    a = []
    for i in range(len(l1)):
        if i in l1 and i in l2:
            a.append((l1.index(i)-l2.index(i))**2)
    return np.mean(a)

In [43]:
#get_network_labelled(species_list2,reac_list,prod_list,background_spc)
def get_network_labelled(species_list2,reac_list,prod_list,background_spc):
    network = np.zeros((len(species_list2),len(species_list2)))
    network_labels = np.zeros((len(species_list2),len(species_list2)))
    for i in range(len(reac_list)):
        for j in range(len(reac_list[i])):
            for k in range(len(prod_list[i])):
                if reac_list[i][j] not in  background_spc and prod_list[i][k] not in  background_spc:
                    network[species_list2.index(reac_list[i][j]),species_list2.index(prod_list[i][k])] = 1
                    network_labels[species_list2.index(reac_list[i][j]),species_list2.index(prod_list[i][k])] = network_labels[species_list2.index(reac_list[i][j]),species_list2.index(prod_list[i][k])]+ [k if k in background_spc else 'none' for k in reac_list[i]]
    return network_labels

In [44]:
#get_reactions(reac_list,prod_list,reactions,reactants,product)
def get_reactions(reac_list,prod_list,reactions,reactants,product):
    reacs = []
    for i in range(len(reac_list)):
        if sorted(reac_list[i]) == sorted(reactants):
            if product in prod_list[i]:
                reacs.append(reactions[i])
    return reacs

In [45]:
#reaction_list_2_eq_list(reaction_list)
def reaction_list_2_eq_list(reaction_list):
    eq_list = []
    for i in reaction_list:
        length = len(i[0])
        eq_string = ''
        coeff = -1
        counter = 0
        while coeff < 0:
            eq_string = eq_string + str(-1*i[0][counter][0])+i[0][counter][0]
            if counter < length-1:
                coeff = i[0][counter + 1][1]
            else:
                break
            if coeff < 0:
                eq_string = eq_string + ' + '
            else:
                eq_string = eq_string + ' = '
            counter = counter + 1
        while counter < len(i[0]):
            eq_string = eq_string + str(-1*i[0][counter][0])+i[0][counter][0]
            counter = counter + 1
            if counter < len(i[0]):
                eq_string = eq_string + ' + '
        eq_list.append(eq_string)
    return eq_list

In [46]:
#eq_list
#list of equations, for full mechanism
eq_list = read_eqns("C:/Users/woods/OneDrive/Documents/Python/isoprene_full_v5_update_1-13-22.txt")

In [47]:
#species_list
#list of species, for full mechanism
species_list = read_spc('/Users/woods/OneDrive/Documents/Python/isoprene_full_v5_spc_update_1-13-22.txt')

In [48]:
species_list.append('MGLYOX')

In [49]:
#background_spc
#a list of background species to be excluded from our reaction network for visual clarity (and also because they will mess with the drg reduction method)
background_spc = ['OH','NO','NO2','NO3','HO2','O3','CH3OO','CO2','CO','O2','N2O5','H2O','H2O2','H2Od','SO2',
                  'H2S','HF','H2S','N2H4','HN3','HI','HBr',
                  'HCl','HCN','H2Se','H2Te','NH2OH','HBrO','HClO','H3PO2','HPO3',
                  'H2O3','OF2','O2F2','NOHSO4','COS','N2F4','N2O4','N2O3','HNO3',
                  'HNO2','N2O','NF5','NI3','H2SO4','CS2','H2CO3','H2SO3','SO2Cl2','S4N4','H2SO5',
                  'H2S2O7','S2F10','H3NO3S','Br2S','SF6','SF4', 'CH3CO3']

In [50]:
reac_list,reac_coeff_list,prod_list,prod_coeff_list = get_prod_reac(eq_list)

In [51]:
network, in_list, out_list = get_network(species_list,reac_list,prod_list,background_spc)

In [52]:
edge_list = []
for i in range(len(out_list)):
    for j in out_list[i]:
        edge_list.append([i,j])

In [53]:
protected2 = ['ISOP1OH23O4OHt','ISOP1OH23O4OHc', 'ISOP1OH2OH34O', 'HCHO', 'GLYC','HAC','PYRAC','MGLY','HCOOH','MACR','MPAN','GLYX','MVK','PAN']

### Yield Estimation Algorithm Functions

In [54]:
#get_scc_fluxes(scc, in_list,out_list,net)
#output:
# [species,[[species,flux],[species,flux]...]]
def get_scc_fluxes(scc, in_list,out_list,net):
    ins_0 = []
    for i in scc:
        for j in in_list[i]:
            if j not in scc:
                ins_0.append(i)
    output = []
    for v in ins_0:
        ins = [v]
        old_list = []
        while len(ins)>0:
            for k in ins:
                k_l = []
                sum1 = 0
                for b in out_list[k]:
                    if b not in old_list:
                        if b not in ins_0:
                            if b in scc:
                                ins.append(b)
                            if k not in out_list[b]:
                                sum1 = sum1 + net[k][b]
                            else:
                                if net[b][k]>0:
                                    sum1 = sum1 + (net[k][b]/net[b][k])*0.0000001
                for c in out_list[k]:
                    if c not in old_list:
                        if c not in ins_0:
                            if k not in out_list[c]:
                                if sum1>0:
                                    k_l.append([c,net[k][c]/sum1])
                                else:
                                    k_l.append([c,0])
                            else:
                                if sum1>0:
                                    if net[c][k]>0:
                                        k_l.append([c,(0.0000001*net[k][c]/net[c][k])/sum1])
                                else:
                                    k_l.append([c,0])
                               
                output.append([k,k_l])
                ins.remove(k)
                old_list.append(k)
    output2 = []
    for z in output:
        if z not in output2:
            output2.append(z)
    return output2 

In [55]:
#get_network_flux(species_list2,reac_list,prod_list,background_spc,conc_list,k_list), weighted network
#creates reaction network of format network[reactant][product]
# in_list gives in_list[species] --> reactants that can react to form this species
# out_list gives in_list[species] --> products that are formed from this species
def get_network_flux(species_list2,reac_list,prod_list,background_spc,conc_list,k_list):
    n1 = time.time()
    network = np.zeros((len(species_list2),len(species_list2)))
    factor = pressure_to_m(1000,295)/1000000000
    for i in range(len(reac_list)):
        bcks = 1
        for p in reac_list[i]:
            if p in background_spc:
                bcks = conc_list[background_spc.index(p)]*factor
        for j in range(len(reac_list[i])):
            for k in range(len(prod_list[i])):
                if reac_list[i][j] not in  background_spc and prod_list[i][k] not in  background_spc:
                    network[species_list2.index(reac_list[i][j]),species_list2.index(prod_list[i][k])] = network[species_list2.index(reac_list[i][j]),species_list2.index(prod_list[i][k])] + k_list[i]*bcks
    x2 = time.time()
    out_list = network_2_out_list(network,species_list2)
    in_list = network_2_in_list(network,species_list2)
    x3 = time.time()
    times = [x2-n1,x3-x2]
    return network, in_list, out_list

In [56]:
#get_network_flux_quick(species_list2,reac_list_n,prod_list_n,bspc,conc_list,k_list)
#creates reaction network of format network[reactant][product]
# in_list gives in_list[species] --> reactants that can react to form this species
# out_list gives in_list[species] --> products that are formed from this species
def get_network_flux_quick(species_list2,reac_list_n,prod_list_n,bspc,conc_list,k_list):
    network = np.zeros((len(species_list2),len(species_list2)))
    factor = pressure_to_m(1000,295)/1000000000
    for i in range(len(reac_list_n)):
        bcks = 1
        for p in reac_list_n[i]:
            if species_list[p] in bspc:
                bcks = conc_list[background_spc.index(species_list[p])]*factor
        for j in reac_list_n[i]:
            for k in prod_list_n[i]:
                if species_list[j] not in bspc and species_list[k] not in bspc:
                    network[j,k] = network[j,k] + k_list[i]*bcks         
    return network

In [57]:
#get_fluxes_init(species_list,reac_list,prod_list,background_spc,conc_mean,k_list)
def get_fluxes_init(species_list,reac_list,prod_list,background_spc,conc_mean,k_list):
    x1 = time.time()
    net,in_list,out_list = get_network_flux(species_list,reac_list,prod_list,background_spc,conc_mean,k_list)
    net2 = deepcopy(net)
    x2 = time.time()
    l = len(net[0])
    for i in range(len(net)):
        s = sum(net[i])
        if s > 0:
            for j in range(l):
                net2[i][j] = net[i][j]/s
        else:
            for j in range(l):
                net2[i][j] = net[i][j]
    x3 = time.time()
    #r_m = get_reachability_matrix(species_list,out_list)
    c_p = get_connected_pairs(out_list)
    cluster_out = edge_list_2_out_list(c_p,species_list)
    scc = get_scc(cluster_out,c_p)
    cycle_flux = []
    x4 = time.time()
    for i in scc:
        cycle_flux.extend(get_scc_fluxes(i, in_list,out_list,net))
    cycle_part = []
    for i in cycle_flux:
        cycle_part.append(i[0])
    new_visit = [0]
    old_list = []
    visit_list = [[] for i in range(len(species_list))]
    flux_list = np.zeros(len(species_list))
    flux_list[0] = 1
    x5 = time.time()
    for i in range(len(species_list)):
        visit_list.append([])
    while len(new_visit)>0:
        for i in new_visit:
            old_list.append(i)
            if i in cycle_part:
                for c in cycle_flux[cycle_part.index(i)][1]:
                    if c[0] not in visit_list[i]:
                        visit_list[i].append(c[0])
                    flux_list[c[0]] = flux_list[c[0]] + c[1]*flux_list[i]
                    if all(x in old_list+cycle_part for x in in_list[c[0]]):
                        new_visit.append(c[0])
            else:              
                for j in out_list[i]:
                    flux_list[j] = flux_list[j] + net2[i][j]*flux_list[i]
                    if j not in visit_list[i]:
                        visit_list[i].append(j)
                    if j not in old_list:
                        if all(x in old_list+cycle_part for x in in_list[j]):
                            new_visit.append(j)
            new_visit.remove(i)
    x6 = time.time()
    times = [x2-x1,x3-x2,x4-x3,x5-x4,x6-x5]
    return flux_list,net,c_p, cycle_part,scc,net2, times

In [58]:
#get_connected_pairs(out_list,in_list,species_list)
def get_connected_pairs(out_list):
    connected_pairs = []
    for i in range(len(out_list)):
        for j in out_list[i]:
            if i in out_list[j]:
                if [i,j] not in connected_pairs:
                    connected_pairs.append([i,j])
    return connected_pairs

In [59]:
#edge_list_2_out_list(edge_list,species_list)
def edge_list_2_out_list(edge_list,species_list):
    out_list = []
    for i in range(len(species_list)):
        out_list.append([])
    for j in edge_list:
        out_list[j[0]].append(j[1])
    return out_list

In [60]:
#get_cluster(species_A,out_list)
def get_cluster(species_A,out_list):
    test_list=[species_A]
    test_list_up = test_list
    visited_list = []
    while len(test_list)>0:
        for i in test_list:
            visited_list.append(i)
            for j in out_list[i]:
                if j not in visited_list:
                    test_list_up.append(j)
            test_list_up.remove(i)
    return visited_list

In [61]:
#get_scc(cluster_out,connected_pairs)
def get_scc(cluster_out,connected_pairs):
    specs = []
    scc = []
    for i in connected_pairs:
        for j in i:
            specs.append(j)
    specs = list(set(specs))
    while len(specs)>0:
        cluster = get_cluster(specs[0],cluster_out)
        for k in cluster:
            if k in specs:
                specs.remove(k)
        scc.append(sorted(cluster))
    return scc

In [62]:
#get_fluxes_quick(species_list,in_list,out_list,scc,c_p,reac_list_n,prod_list_n,bspc,conc_list,k_list)
def get_fluxes_quick_groups(species_list,in_list,out_list,scc,c_p,reac_list_n,prod_list_n,bspc,conc_list,k_list,groups,edge_list, reaction_species):
    net = get_network_flux_quick(species_list,reac_list_n,prod_list_n,bspc,conc_list,k_list)
    np.seterr('raise')
    ls = len(species_list)
    net2 = normalize(net, norm='l1', axis=1)
    cycle_flux = []
    for i in scc:
        cycle_flux.extend(get_scc_fluxes(i, in_list,out_list,net))
    cycle_part = [i[0] for i in cycle_flux]
    cycle_part_set = set(cycle_part)
    new_visit = [0]
    old_list = set([])
    visit_list = [[] for i in range(ls)]
    flux_list = np.zeros(ls)
    flux_list[0] = 1
    flux_groups = np.zeros(len(groups))
    net_flux = np.zeros((len(species_list),len(species_list)))
    while len(new_visit)>0:
        for i in new_visit:
            old_list.add(i)
            oc = old_list.union(cycle_part_set)
            if i in cycle_part_set:
                for c in cycle_flux[cycle_part.index(i)][1]:
                    f = c[1]*flux_list[i]
                    if c[0] not in visit_list[i]:
                        visit_list[i].append(c[0])
                    flux_list[c[0]] = flux_list[c[0]] + f
                    for g in range(len(groups)):
                        if c[0] in groups[g] and i not in groups[g]:
                            flux_groups[g] = flux_groups[g] + f
                    if all(x in oc for x in in_list[c[0]]):
                        new_visit.append(c[0])
            else:              
                for j in out_list[i]:
                    f = net2[i][j]*flux_list[i]
                    flux_list[j] = flux_list[j] + f
                    for g in range(len(groups)):
                        if j in groups[g] and i not in groups[g]:
                            flux_groups[g] = flux_groups[g] + f
                    if j not in visit_list[i]:
                        visit_list[i].append(j)
                    if j not in old_list:
                        if all(x in oc for x in in_list[j]):
                            new_visit.append(j)
            new_visit.remove(i)
    bckg_fluxes = np.zeros(len(background_spc))
    factor = pressure_to_m(1000,295)/1000000000
    rates = [np.prod(np.array([flux_list[m] if species_list[m] not in bspc else 1 for m in reac_list_n[n]])) for n in range(len(reac_list))]
    k_factors = [sum([rates_0[i] for i in j]) for j in reaction_species]
    k_factor_list = []
    for i in range(len(reac_list)):
        for j in reac_list[i]:
            if j not in bspc:
                val = k_factors[species_list.index(j)]
                if val !=0:
                    k_factor = rates_0[i]/k_factors[species_list.index(j)]
                else:
                    k_factor = 0
                k_factor_list.append(k_factor)
        for j in reac_list[i]:
            if j in bspc:
                ind = background_spc.index(j)
                bckg_fluxes[ind] = bckg_fluxes[ind] + rates[i]*bckg_coeff[i][ind]*k_factor
    return cycle_flux,flux_list, flux_groups, bckg_fluxes

In [63]:
def get_delt(z1,z2):
    return np.mean([abs(z1[0][i] - z2[0][i])/n[i] for i in range(len(z[0][0]))])


In [64]:
def get_delt(z1,z2):
    return np.mean([abs(z1[0][i] - z2[0][i])/n[i] for i in range(len(z[0][0]))])


In [65]:
species_list.index('ISOP1OH23O4OHt')

122

In [66]:
prod_list

[['ISOP1OHc'],
 ['ISOP1OHt'],
 ['ISOP4OHc'],
 ['ISOP4OHt'],
 ['ISOP1OH4OOt'],
 ['ISOP1OH2OO'],
 ['ISOP1OH2OO'],
 ['ISOP1OH4OOc'],
 ['ISOP1OO4OHt'],
 ['ISOP3OO4OH'],
 ['ISOP3OO4OH'],
 ['ISOP1OO4OHc'],
 ['ISOP1OHt'],
 ['ISOP1OHt'],
 ['ISOP1OHc'],
 ['ISOP1OHc'],
 ['ISOP4OHt'],
 ['ISOP4OHt'],
 ['ISOP4OHc'],
 ['ISOP4OHc'],
 ['NO2', 'MVK', 'HO2', 'HCHO'],
 ['ISOP1OH2N'],
 ['NO2', 'HO2', 'ISOP1CO4OH'],
 ['NO2', 'ISOP1CO2OO3OOH4OH'],
 ['ISOP1OH4Nc'],
 ['NO2', 'HO2', 'ISOP1CO4OH'],
 ['NO2', 'ISOP1CO2OO3OOH4OH'],
 ['ISOP1OH4Nt'],
 ['ISOP1CO2OOH3OO4OH'],
 ['ISOP1CO2OO3OOH4OH'],
 ['NO2', 'MGLY', 'GLYC', 'OH'],
 ['ISOP1CO2N3OOH4OH'],
 ['MVK3OOH4OH', 'CO', 'OH'],
 ['OH', 'GLYC', 'MGLY'],
 ['NO2', 'MGLY', 'GLYC', 'OH'],
 ['ISOP1CO2OOH3N4OH'],
 ['MVK3OOH4OH', 'CO', 'OH'],
 ['ISOP1CO2OOH3OOH4OH'],
 ['OH', 'GLYC', 'MGLY'],
 ['NO2', 'MACR', 'HO2', 'HCHO'],
 ['ISOP3N4OH'],
 ['NO2', 'HO2', 'ISOP1OH4CO'],
 ['NO2', 'ISOP1OH2OOH3OO4CO'],
 ['ISOP1N4OHc'],
 ['NO2', 'HO2', 'ISOP1OH4CO'],
 ['NO2', 'ISOP1OH2OOH3OO

In [67]:
z

NameError: name 'z' is not defined

In [78]:
conc_list_post = [#'OH','NO','NO2','NO3','HO2','O3','CH3OO',
                 420,0,210000000,0,0.01,0,0.01,0,
                  0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0,0,0,0,0,0,
                  0,0,0,0,0,0, 0]

In [79]:
#'OH','NO','NO2','NO3','HO2','O3','CH3OO'
conc_list_input = [1e-5,1e-5,1e-5,1e-5,0.01,1,0.001]

In [80]:
CFACTOR = 2.5e+19;
RO2 = 0.01*10e-9;
SUN = 1;
TEMP = 295;
M = pressure_to_m(1000,295);
sza = 0;

In [81]:
#klist
x1 = time.time()
k_list = []
for i in eq_list:
    if i[1] == 'J(22) ':
        I = 5.804e-6;
        m = 1.092;
        n = 0.377;
        k_list.append(j_func(sza,I,m,n))
    elif i[1] == 'J(34) ':
        I = 1.537e-4;
        m = 0.170;
        n = 0.208;
        k_list.append(j_func(sza,I,m,n))
    elif i[1] == 'J(41) ':
        I = 7.649e-6;
        m = 0.682;
        n = 0.279;
        k_list.append(j_func(sza,I,m,n))
    else:
        k_list.append(eval(i[1])) 
x2 = time.time()

In [82]:
conc_list = conc_list_input
conc_list.extend(conc_list_post)

In [83]:
# if reactant in group, dont add flux to group, if reactant not in group and product in group, add flux to group

In [84]:
# a list of background species to be excluded from our reaction network for visual clarity (and also because they will mess with the drg reduction method)
background_spc = ['OH','NO','NO2','NO3','HO2','O3','CH3OO','CO2','CO','O2','N2O5','H2O','H2O2','H2Od','SO2',
                  'H2S','HF','H2S','N2H4','HN3','HI','HBr',
                  'HCl','HCN','H2Se','H2Te','NH2OH','HBrO','HClO','H3PO2','HPO3',
                  'H2O3','OF2','O2F2','NOHSO4','COS','N2F4','N2O4','N2O3','HNO3',
                  'HNO2','N2O','NF5','NI3','H2SO4','CS2','H2CO3','H2SO3','SO2Cl2','S4N4','H2SO5',
                  'H2S2O7','S2F10','H3NO3S','Br2S','SF6','SF4']

In [85]:
bckg_fluxes = np.zeros(len(background_spc))

In [86]:
bckg_coeff = [np.zeros(len(background_spc)) for i in reac_list]

In [87]:
bspc = set(background_spc)

In [88]:
reaction_list = eqn_to_reactions2(reac_list,reac_coeff_list,prod_list,prod_coeff_list,eq_list)

In [89]:
CFACTOR = 2.5e+19;
RO2 = 0.01*10e-9;
SUN = 1;
TEMP = 295;
M = pressure_to_m(1000,295);
sza = 0;

In [90]:
prod_list_n = [[species_list.index(j) for j in i] for i in prod_list]
reac_list_n = [[species_list.index(j) for j in i] for i in reac_list]

In [91]:
for i in range(len(reaction_list)):
    for j in reaction_list[i][0]:
        if j[0] in bspc:
            bckg_coeff[i][background_spc.index(j[0])] = j[1]

In [92]:
reaction_species = [[] for i in range(len(species_list))]
for i in range(len(reac_list_n)):
    if any([x for x in reac_list[i]]) in bspc:
        for j in reac_list_n[i]:
            reaction_species[j].append(i)

In [93]:
reaction_species = [[] for i in range(len(species_list))]
for i in range(len(reac_list_n)):
    for j in reac_list_n[i]:
        reaction_species[j].append(i)

In [94]:
rates_0 = [k_list[n]*np.prod(np.array([conc_list[background_spc.index(species_list[m])] if species_list[m] in bspc else 1 for m in reac_list_n[n]])) for n in range(len(reac_list))]
k_factors = [sum([rates_0[i] for i in j]) for j in reaction_species]

In [95]:
g1 = [122,123,169]
g2 = [67,69,71]

In [96]:
flux_list,net,c_p, cycle_part,scc,net2, times = get_fluxes_init(species_list,reac_list,prod_list,background_spc,conc_list,k_list)

In [97]:
org_nit = ['ISOP1CO2N3OOH4OH', 'ISOP1CO2OOH3N4OH', 'ISOP1OH2OOH3N4CO', 'ISOP1OH2N3OOH4CO', 'ISOP1OH2N3CO4OH', 'ISOP1CO2N3OOH4OOH', 'ISOP1CO2OOH3N4OOH', 'ISOP1CO2OOH3OOH4N', 'ISOP1OOH2OOH3N4CO', 'ISOP1OOH2N3OOH4CO', 'ISOP1N2OOH3OOH4CO', 'ISOP1OOH23O3OH4N', 'ISOP1N2OOH34O4OH', 'ISOP1OH12O3OOH4N', 'ISOP1OH2OOH3OH4N', 'ISOP1OH2N3OH4OOH', 'ISOP1N2OH3OOH4OH', 'ISOP1OOH2OH3N4OH', 'ISOP1OH2OOH3N4OH', 'ISOP1OH2N3OOH4OH', 'ISOP1N2OOH3OH4CO', 'ISOP1CO2OH3OOH4N', 'ISOP1OOH2OH3CO4N', 'ISOP1N4R4OH', 'ISOP1OH1R4N', 'ISOP3OH3R4N', 'ISOP1OOH2R3CO4N', 'ISOP1OH2OH3N4CO', 'ISOP1CO2N3OH4OH', 'ISOP1OH2N3R4OH', 'ISOP1OH2N3OH4N', 'ISOP1OH2N3N4OH', 'ISOP1CO2N3OH4OOH', 'ISOP1OH2R3N4OH', 'ISOP1N2OH3N4OH', 'ISOP1OOH2OH3N4CO', 'ISOP1N2OOH', 'ISOP1N4OOH', 'ISOP3OOH4N', 'ISOP1OOH4N', 'ISOP1N2OH', 'ISOP3CO4N', 'ISOP1CO4N', 'ISOP1O4N', 'ISOP1N4CO', 'ISOP1N4O', 'ISOP3OH4N', 'ISOP1N2N', 'ISOP1N4N', 'ISOP3N4N', 'ISOP1N253O4OH', 'ISOP1N253N4OH', 'ISOP1N253OOH4OH', 'ISOP1N253CO4OH', 'ISOP1N253OH4OH', 'ISOP1N2OOH3R4OH', 'ISOP1N23O4OH', 'ISOP1N2OH3R4OOH', 'ISOP1N2OH34O', 'ISOP1N2R3OH4OOH', 'ISOP1OH2R3OOH4N', 'ISOP1OH23O4N', 'ISOP1OOH2R3OH4N', 'ISOP12O3OH4N', 'ISOP1OOH2OH3R4N', 'ISOP1N2OOH3N4OH', 'ISOP1N2OOH3OOH4OH', 'ISOP1N2OOH3OH4N', 'ISOP1N2OOH3OH4OOH', 'ISOP1N2OH3N4OOH', 'ISOP1N2OH3OOH4OOH', 'ISOP1N2N3OH4OOH', 'ISOP1OH2N3OOH4N', 'ISOP1OH2OOH3OOH4N', 'ISOP1N2OH3OOH4N', 'ISOP1OOH2OH3OOH4N', 'ISOP1OOH2N3OH4N', 'ISOP1OOH2OOH3OH4N', 'ISOP1OO2OOH3OH4N', 'ISOP1N2N3OOH4OH', 'ISOP1OH2OOH3N4N', 'ISOP1N23O4OH4R', 'ISOP1N23O4CO', 'ISOP1OH1R23O4N', 'ISOP1CO23O4N', 'ISOP1N2OH3CO4OOH', 'ISOP12O3OH3R4N', 'ISOP12O3CO4N', 'ISOP1N2OH3CO4N', 'ISOP1OH2OOH3CO4N', 'ISOP1N4R4CO', 'ISOP1N4CO4OOH', 'ISOP1N4CO4OH', 'ISOP1CO1R4N', 'ISOP1CO1OOH4N', 'ISOP1CO1OH4N', 'ISOP1N2R3OH4CO', 'ISOP1CO2OH3R4N', 'ISOP1N2OH3OOH4CO', 'ISOP1CO2OOH3OH4N', 'ISOP1N2R3OH4OH', 'ISOP1OH2OH3R4N', 'ISOP1N2OOH3OH4OH', 'ISOP1OH2OH3OOH4N', 'ISOP1N2OH3OH4OOH', 'ISOP1OOH2OH3OH4N', 'ISOP1OOH2OH3N4N', 'ISOP1OH2N3CO4N', 'ISOP1N2N3CO4OH', 'ISOP1OH2OH3N4N', 'ISOP1N2OH3OH4N', 'ISOP1N2N3OH4OH', 'ISOP1CO2N3OH4N', 'ISOP1N2OH3N4CO', 'ISOP1N4PAN', 'ISOP1PAN4N','ISOP1PAN4OH','ISOP1OH4PAN']

In [98]:
#df2 = pd.DataFrame(bbn)

In [99]:
#df2.to_csv('/Users/woods/Documents/Documents - Woods MacBook Air/Jobs/Columbia/McNeill Lab/Woods_AMORE_code/cyc.csv')

In [123]:
#new x
xOH = [1e-6,1e-4]
xNO = [1.17e-6,5.32e-1]
xNO2 = [1.01e-4,1.01e-2]
xNO3 = [2.3e-4,2e-2]
xHO2 = [4.15e-2,0.3]
xO3 = [16.7,100]
xCH3OO = [0.1,0.2]
xT = [292,292]
xP = [1000,1000]
xSol = [0.,1]

In [112]:
#new x
xOH = [1e-6,1e-4]
xNO = [1.17e-6,1.32e-1]
xNO2 = [1.01e-4,1.01e-2]
xNO3 = [2.3e-4,2e-3]
xHO2 = [4.15e-2,0.3]
xO3 = [16.7,50]
xCH3OO = [1e-4,0.2]
xT = [300,300]
xP = [1000,1000]
xSol = [0,0.1]

In [124]:
x_list = []
for i in xOH:
    for j in xNO:
        for k in xNO2:
            for l in xNO3:
                for m in xHO2:
                    for n in xO3:
                        for o in xCH3OO:
                            for p in xT:
                                for q in xP:
                                    for r in xSol:
                                        x_list.append([i,j,k,l,m,n,o,p,q,r])

In [125]:
k_lists = [[[[],[]],[[],[]]],[[[],[]],[[],[]]]]
count = 0
for a in [0,1]:
    for b in [0,1]:
        for c in [0,1]:
            CFACTOR = 2.5e+19;
            RO2 = 0.01*10e-9;
            SUN = xSol[c];
            TEMP = xT[a];
            M = pressure_to_m(xP[b],xT[a]);
            sza = 0;
            k_list = []
            for i in eq_list:
                if i[1] == 'J(22) ':
                    I = 5.804e-6;
                    m = 1.092;
                    n = 0.377;
                    k_list.append(j_func(sza,I,m,n))
                elif i[1] == 'J(34) ':
                    I = 1.537e-4;
                    m = 0.170;
                    n = 0.208;
                    k_list.append(j_func(sza,I,m,n))
                elif i[1] == 'J(41) ':
                    I = 7.649e-6;
                    m = 0.682;
                    n = 0.279;
                    k_list.append(j_func(sza,I,m,n))
                else:
                    k_list.append(eval(i[1])) 
            count = count + 1
            k_lists[a][b][c] = k_list
            

In [126]:
protected2 = ['ISOP1OH23O4OHt','ISOP1OH23O4OHt',
 'ISOP1OH23O4OHc','ISOP1OH23O4OHc',
 'ISOP1OH2OH34O', 'ISOP1OH2OH34O',
 'HCHO','HCHO','HCHO',
 'GLYC',
 'HAC',
 'PYRAC',
 'MGLY',
 'HCOOH',
 'MACR',
 'MPAN',
 'GLYX',
 'MVK']

In [127]:
protected2 = ['ISOP1OH23O4OHt',
 'ISOP1OH23O4OHc',
 'ISOP1OH2OH34O',
 'HCHO',
 'GLYC',
 'HAC',
 'PYRAC',
 'MGLY',
 'HCOOH',
 'MACR',
 'MPAN',
 'GLYX',
 'MVK']

In [128]:
xx = time.time()
z = []
for i in range(1000):
    conc_list_input = x_list[i][0:7]
    conc_list = conc_list_input
    conc_list.extend(conc_list_post)
    k_list_now = k_lists[xT.index(x_list[i][7])][xP.index(x_list[i][8])][xSol.index(x_list[i][9])]
    a,b,cc,dd = get_fluxes_quick_groups(species_list,in_list,out_list,scc,c_p,reac_list_n,prod_list_n,bspc,conc_list,k_list_now,[g1,g2],edge_list,reaction_species)
    f = [b[species_list.index(i)] for i in protected2]
    fnit = [b[species_list.index(i)] for i in org_nit]
    f.append(sum(fnit))
    #f.append(sum(fnit))
    #f.append(sum(fnit))
    #z.append([f, cc, dd])
    z.append([f])
xy = time.time() 
print(xy-xx)

26.042194604873657


In [129]:
spec = [[] for i in range(len(z[1][0]))]
for i in z:
    for j in i:
        for k in range(len(j)):
            spec[k].append(j[k])

In [130]:
n = [max(i) for i in spec]

In [131]:
n

[0.19920299543393782,
 0.09576710268162678,
 0.0009643754772037256,
 1.5026419199064844,
 3.849360707565448,
 11.249158534971896,
 0.18982142050875864,
 0.9042082665579319,
 0.019877161139686142,
 0.32234031151497833,
 7.473867735472757e-06,
 0.5816114268855229,
 0.40695827875977336,
 2.5167356127483167]

In [132]:
delta = []
ones_list = []
for i in range(len(z)):
    x_inds = [xOH.index(x_list[i][0]),xNO.index(x_list[i][1]),xNO2.index(x_list[i][2]),xNO3.index(x_list[i][3]),xHO2.index(x_list[i][4]),xO3.index(x_list[i][5]),xCH3OO.index(x_list[i][6]),xT.index(x_list[i][7]),xP.index(x_list[i][8]),xSol.index(x_list[i][9])]
    ones = []
    for j in range(len(x_inds)):
        if x_inds[j] ==1:
            ones.append(j)
    ones_list.append(ones)
    condishes = []
    for j in ones:
        c = deepcopy(x_inds)
        c[j] = 0
        condishes.append(c)
    deltas = [1]
    for k in condishes:
        index = x_list.index([xOH[k[0]],xNO[k[1]],xNO2[k[2]],xNO3[k[3]],xHO2[k[4]],xO3[k[5]],xCH3OO[k[6]],xT[k[7]],xP[k[8]],xSol[k[9]]])
        deltas.append(get_delt(z[i],z[index]))
    delta.append(min(deltas))

In [135]:
thresh = 0.12
groups = []
for i in range(len(delta)):
    if delta[i] > thresh:
        if ones_list[i] not in groups:
            groups.append(ones_list[i])
len(groups),groups

(13,
 [[],
  [9],
  [5],
  [3],
  [3, 9],
  [3, 4, 9],
  [1],
  [1, 3],
  [0],
  [0, 3],
  [0, 3, 9],
  [0, 1],
  [0, 1, 3]])

In [ ]:
ones_list.index([5])

In [ ]:
z[16]

In [ ]:
protected2

In [ ]:
z[768]

In [ ]:
(15,
 [[],
 x [9],
  [5],
  [3],
 x [3, 9],
  [3, 4],
 x [3, 4, 9],
 x [1],
  [1, 3],
  [0],
  [0, 4],
  [0, 3],
 x [0, 3, 9],
  [0, 1],
 x [0, 1, 3]])

In [ ]:
ones_list.index([3])

## Yield Estimation Algorithm Assessment

In [ ]:
#new x
xOH = [1e-6,1e-4]
xNO = [1.17e-6,5.32e-1]
xNO2 = [1.01e-4,1.01e-2]
xNO3 = [2.3e-4,2e-2]
xHO2 = [4.15e-2,0.3]
xO3 = [16.7,100]
xCH3OO = [0.1,0.2]
xT = [290,290.1]
xP = [900,900.1]
xSol = [0,1]

In [ ]:
x_list = []
for i in xOH:
    for j in xNO:
        for k in xNO2:
            for l in xNO3:
                for m in xHO2:
                    for n in xO3:
                        for r in xSol:
                            x_list.append([i,j,k,l,m,n,r])

In [ ]:
matlab_input = pd.DataFrame(x_list)

In [ ]:
matlab_input.to_csv("C:/Users/woods/OneDrive/Documents/MATLAB/yield_estimation_algo_data")

In [ ]:
protected2

In [ ]:
x_list[1]

In [68]:
protected3  = ['HCHO',"MVK","GLYX","MACR","MGLY","PAN","ISOP1OH23O4OHt","ISOP1OH23O4OHc","ISOP1OH2OH34O"]

In [69]:
background_spc

['OH',
 'NO',
 'NO2',
 'NO3',
 'HO2',
 'O3',
 'CH3OO',
 'CO2',
 'CO',
 'O2',
 'N2O5',
 'H2O',
 'H2O2',
 'H2Od',
 'SO2',
 'H2S',
 'HF',
 'H2S',
 'N2H4',
 'HN3',
 'HI',
 'HBr',
 'HCl',
 'HCN',
 'H2Se',
 'H2Te',
 'NH2OH',
 'HBrO',
 'HClO',
 'H3PO2',
 'HPO3',
 'H2O3',
 'OF2',
 'O2F2',
 'NOHSO4',
 'COS',
 'N2F4',
 'N2O4',
 'N2O3',
 'HNO3',
 'HNO2',
 'N2O',
 'NF5',
 'NI3',
 'H2SO4',
 'CS2',
 'H2CO3',
 'H2SO3',
 'SO2Cl2',
 'S4N4',
 'H2SO5',
 'H2S2O7',
 'S2F10',
 'H3NO3S',
 'Br2S',
 'SF6',
 'SF4',
 'CH3CO3']

In [70]:
conc_list_post = [0.01, 420, 0, 210000000, 0, 0.01, 0,0.01,0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0, 0, 0]

In [71]:
xx = time.time()
z = []
for i in range(128):
    conc_list_input = x_list[i][0:6]
    conc_list = conc_list_input
    conc_list.extend(conc_list_post)
    k_list_now = k_lists[0][0][xSol.index(x_list[i][6])]
    a,b,cc,dd = get_fluxes_quick_groups(species_list,in_list,out_list,scc,c_p,reac_list_n,prod_list_n,bspc,conc_list,k_list_now,[g1,g2],edge_list,reaction_species)
    f = [b[species_list.index(i)] for i in protected3]
    fnit = [b[species_list.index(i)] for i in org_nit]
    f.append(sum(fnit))
    #f.append(sum(fnit))
    #f.append(sum(fnit))
    #z.append([f, cc, dd])
    z.append([f])
xy = time.time() 
print(xy-xx)

NameError: name 'x_list' is not defined

In [ ]:
sum(fnit)

In [72]:
yield_estimator_output = pd.DataFrame(z)

In [73]:
yield_estimator_output.to_csv("C:/Users/woods/OneDrive/Documents/MATLAB/yield_estimation_compare_data3")

In [74]:
z

[]

In [75]:
protected3

['HCHO',
 'MVK',
 'GLYX',
 'MACR',
 'MGLY',
 'PAN',
 'ISOP1OH23O4OHt',
 'ISOP1OH23O4OHc',
 'ISOP1OH2OH34O']

In [76]:
x_list[0]

NameError: name 'x_list' is not defined

In [77]:
x_list[1]

NameError: name 'x_list' is not defined